In [69]:
from database.engine import session_maker
from database.models import Municipalities, MunicSubscriptions, DFloodAggoAndChs, DFloodKrudor, FCategoriesSubscriptions
from sqlalchemy import select, text
from telebot import TeleBot
from config import BOT_TOKEN
import pandas as pd
from icecream import ic

In [4]:
bot = TeleBot(BOT_TOKEN)

In [ ]:
def split_message(message, max_length=3000):
    if len(message) <= max_length:
        return [message]

    messages = []
    while len(message) > max_length:
        split_index = message[:max_length].rfind('\n\n')
        if split_index == -1:
            split_index = max_length

        messages.append(message[:split_index])
        message = message[split_index:].lstrip()

    if message:
        messages.append(message)
    return messages



def decline_phrase(n):
    forms_section = ['участке', 'участках', 'участке']
    forms_road = ['дороги', 'дорог', 'дорог']

    if 10 <= n % 100 <= 20:
        section_form = forms_section[1]
    else:
        last_digit = n % 10
        if last_digit == 1:
            section_form = forms_section[0]
        elif 2 <= last_digit <= 4:
            section_form = forms_section[2]
        else:
            section_form = forms_section[1]

    if 10 <= n % 100 <= 20:
        road_form = forms_road[2]
    else:
        last_digit = n % 10
        if last_digit == 1:
            road_form = forms_road[0]
        else:
            road_form = forms_road[2]

    return f"на {n} {section_form} {road_form}"


def response_creator(df):
    response = ''
    df['danger_icon'] = df.apply(lambda row: '🔴' if row['oper_mode'] in ('Чрезвычайная ситуация', 'Режим повышенной готовности') else '🟢', axis=1)
    df['date_event'] = df['date_event'].apply(lambda x: x.strftime('%d.%m.%Y') if pd.notnull(x) else x)
    df['f_closing_date'] = df['f_closing_date'].apply(lambda x: x.strftime('%d.%m.%Y') if pd.notnull(x) else x)
    df['f_opening_date'] = df['f_opening_date'].apply(lambda x: x.strftime('%d.%m.%Y') if pd.notnull(x) else x)
	
    for i in df.iterrows():
        declined_roads = decline_phrase(int(i[1]['f_road_q']))
		
        response += f"{i[1]['danger_icon']} {i[1]['municipality']} " + \
					f"{i[1]['date_event']}.\n"
        if i[1]['danger_icon'] == '🔴':
            response += "ПЕРЕЛИВ ДОРОГИ. "

        response += f"{i[1]['type_flood']} \n" +\
		f"А/Д: {i[1]['road']}. Местоположение {i[1]['f_location']}\n" + \
		f"Общая протяженность: {i[1]['f_road_l']} м на {declined_roads} \n" + \
		f"Текущий уровень воды над проезжей частью: {i[1]['f_water_level']} см\n" + \
		f"Дорога закрыта с {i[1]['f_closing_date']}\n"
        if i[1]['f_detour'] == 'Да':
            response += "Есть объездная дорога\n"
        else:
            response += "Нет объездной дороги\n"
		
        response += f"{i[1]['oper_mode']}. "
        if i[1]['danger_icon'] == '🟢':
            response += "НЕТ УГРОЗЫ\n\n"
        else:
            response += "\n\n"

    return response

In [84]:
async with session_maker() as session_:
    query = text("""SELECT flood.d_flood_krudor.municipality, flood.d_flood_krudor.date_event,
 				flood.d_flood_krudor.type_flood,
 				flood.d_flood_krudor.road, flood.d_flood_krudor.f_location,
 				flood.d_flood_krudor.f_road_l, flood.d_flood_krudor.f_road_q,
     			flood.d_flood_krudor.f_water_level,  flood.d_flood_krudor.f_closing_date,
        		flood.d_flood_krudor.f_opening_date, flood.d_flood_krudor.oper_mode, flood.d_flood_krudor.f_detour FROM flood.d_flood_krudor ORDER BY flood.d_flood_krudor.date_create DESC LIMIT 1""")
    response = await session_.execute(query)
    flood_districts = response.all()
    
    df1 = pd.DataFrame(flood_districts)
    
    
    query_subscriptions = text("""SELECT shared.r_omsu.caption_full, flood.d_tg_bot_municip_subscriptions.municipality_id,
 				flood.d_tg_bot_municip_subscriptions.user_id FROM flood.d_tg_bot_municip_subscriptions
     			JOIN shared.r_omsu ON shared.r_omsu.id_r_omsu = flood.d_tg_bot_municip_subscriptions.municipality_id""")
        
    ic(df1.columns)
    response = await session_.execute(query_subscriptions)
    subscriptions = response.all()
    df2 = pd.DataFrame(subscriptions)
    
    ic(df2.columns)
    df = df1.merge(df2, how='inner', left_on=df1['municipality'], right_on=df2['caption_full'])
    ic(df)
    df_grouped = df.groupby('user_id')
    for user_id, group in df_grouped:
        response = response_creator(group)
        try:
            bot.send_message(user_id, text=response)
        except Exception as e:
            print('error:', e)
    
   


ic| df1.columns: Index(['municipality', 'date_event', 'type_flood', 'road', 'f_location',
                        'f_road_l', 'f_road_q', 'f_water_level', 'f_closing_date',
                        'f_opening_date', 'oper_mode', 'f_detour'],
                       dtype='object')
ic| df2.columns: Index(['caption_full', 'municipality_id', 'user_id'], dtype='object')
ic| df:                 key_0        municipality  date_event type_flood  \
        0  Партизанский район  Партизанский район  2024-12-02       Иное   
        
                        road f_location  f_road_l  f_road_q f_water_level  \
        0  Партизанское-Мина     64+060       0.3         1            16   
        
          f_closing_date f_opening_date              oper_mode f_detour  \
        0     2024-11-26           None  Чрезвычайная ситуация       Да   
        
                 caption_full  municipality_id    user_id  
        0  Партизанский район               47  964635576  
/var/folders/wh/05qdgscx4r54_d

In [38]:
df.query('municipality in@response_muns')

,municipality,date_event,type_flood,road,f_location,f_road_l,f_road_q,f_water_level,f_closing_date,f_opening_date,oper_mode,f_detour,danger_icon
0,Партизанский район,28.11.2024,Иное,Партизанское-Мина,64+060,1.34,1,38,26.11.2024,None,Чрезвычайная ситуация,Да,🔴
1,Партизанский район,26.11.2024,Иное,Партизанское-Мина,64+060,1.34,1,40,26.11.2024,None,Чрезвычайная ситуация,Да,🔴
2,Партизанский район,27.11.2024,Иное,Партизанское-Мина,64+060,1.34,1,40,26.11.2024,None,Чрезвычайная ситуация,Да,🔴
3,Партизанский район,29.11.2024,Иное,Партизанское-Мина,64+060,1.34,1,15,26.11.2024,None,Чрезвычайная ситуация,Да,🔴
4,Партизанский район,02.12.2024,Иное,Партизанское-Мина,64+060,0.30,1,16,26.11.2024,None,Чрезвычайная ситуация,Да,🔴
5,Партизанский район,30.11.2024,Иное,Партизанское-Мина,64+060,0.30,1,18-23,26.11.2024,None,Чрезвычайная ситуация,Да,🔴
6,Партизанский район,01.12.2024,Иное,Партизанское-Мина,64+060,0.30,1,18-23,26.11.2024,None,Чрезвычайная ситуация,Да,🔴


In [36]:
ic(df)

ic| df:          municipality  date_event type_flood               road f_location  \
        0  Партизанский район  28.11.2024       Иное  Партизанское-Мина     64+060   
        1  Партизанский район  26.11.2024       Иное  Партизанское-Мина     64+060   
        2  Партизанский район  27.11.2024       Иное  Партизанское-Мина     64+060   
        3  Партизанский район  29.11.2024       Иное  Партизанское-Мина     64+060   
        4  Партизанский район  02.12.2024       Иное  Партизанское-Мина     64+060   
        5  Партизанский район  30.11.2024       Иное  Партизанское-Мина     64+060   
        6  Партизанский район  01.12.2024       Иное  Партизанское-Мина     64+060   
        
           f_road_l  f_road_q f_water_level f_closing_date f_opening_date  \
        0      1.34         1            38     26.11.2024           None   
        1      1.34         1            40     26.11.2024           None   
        2      1.34         1            40     26.11.2024           Non

,municipality,date_event,type_flood,road,f_location,f_road_l,f_road_q,f_water_level,f_closing_date,f_opening_date,oper_mode,f_detour,danger_icon
0,Партизанский район,28.11.2024,Иное,Партизанское-Мина,64+060,1.34,1,38,26.11.2024,None,Чрезвычайная ситуация,Да,🔴
1,Партизанский район,26.11.2024,Иное,Партизанское-Мина,64+060,1.34,1,40,26.11.2024,None,Чрезвычайная ситуация,Да,🔴
2,Партизанский район,27.11.2024,Иное,Партизанское-Мина,64+060,1.34,1,40,26.11.2024,None,Чрезвычайная ситуация,Да,🔴
3,Партизанский район,29.11.2024,Иное,Партизанское-Мина,64+060,1.34,1,15,26.11.2024,None,Чрезвычайная ситуация,Да,🔴
4,Партизанский район,02.12.2024,Иное,Партизанское-Мина,64+060,0.30,1,16,26.11.2024,None,Чрезвычайная ситуация,Да,🔴
5,Партизанский район,30.11.2024,Иное,Партизанское-Мина,64+060,0.30,1,18-23,26.11.2024,None,Чрезвычайная ситуация,Да,🔴
6,Партизанский район,01.12.2024,Иное,Партизанское-Мина,64+060,0.30,1,18-23,26.11.2024,None,Чрезвычайная ситуация,Да,🔴


In [59]:
async with session_maker() as session_:
    query = select(FCategoriesSubscriptions.user_id).where(FCategoriesSubscriptions.category_id==1)
    response = await session_.execute(query)
    result = response.all()
    response = [i[0] for i in result]
    ic(response)

ic| response: [964635576, 5685044175]


In [60]:
async with session_maker() as session_:
    query = select(Municipalities.caption_full, MunicSubscriptions.municipality_id, MunicSubscriptions.user_id).join(MunicSubscriptions, Municipalities.id_r_omsu ==MunicSubscriptions.municipality_id)
    response = await session_.execute(query)
    result = response.all()
    ic(result)
    # response_muns = [i[0] for i in result]
    # ic(response_muns)

ic| result: [('Казачинский район', 34, 964635576),
             ('Канский район', 35, 964635576),
             ('Каратузский район', 36, 964635576),
             ('Кежемский район', 37, 964635576),
             ('Козульский район', 38, 964635576),
             ('Краснотуранский район', 39, 964635576),
             ('Курагинский район', 40, 964635576),
             ('Манский район', 41, 964635576),
             ('Минусинский район', 42, 964635576),
             ('Мотыгинский район', 43, 964635576),
             ('Назаровский район', 44, 964635576),
             ('Нижнеингашский район', 45, 964635576),
             ('Новоселовский район', 46, 964635576),
             ('Партизанский район', 47, 964635576),
             ('Пировский муниципальный округ', 48, 964635576),
             ('Рыбинский район', 49, 964635576),
             ('Саянский район', 50, 964635576),
             ('Северо-Енисейский район', 51, 964635576),
             ('Сухобузимский район', 52, 964635576),
             ('Та

In [61]:
df2 = pd.DataFrame(result)
df2

,caption_full,municipality_id,user_id
0,Казачинский район,34,964635576
1,Канский район,35,964635576
2,Каратузский район,36,964635576
3,Кежемский район,37,964635576
4,Козульский район,38,964635576
...,...,...,...
59,Иланский район,32,964635576
60,Ирбейский район,33,964635576
61,город Бородино,3,5685044175
62,город Дивногорск,4,5685044175


In [62]:
df2_grouped = df2.groupby('user_id')
for user_id, group in df2_grouped:
    ic(group)

ic| group:          caption_full  municipality_id    user_id
           0   Казачинский район               34  964635576
           1       Канский район               35  964635576
           2   Каратузский район               36  964635576
           3     Кежемский район               37  964635576
           4    Козульский район               38  964635576
           ..                ...              ...        ...
           56   Енисейский район               29  964635576
           57  Ермаковский район               30  964635576
           58    Идринский район               31  964635576
           59     Иланский район               32  964635576
           60    Ирбейский район               33  964635576
           
           [61 rows x 3 columns]
ic| group:         caption_full  municipality_id     user_id
           61    город Бородино                3  5685044175
           62  город Дивногорск                4  5685044175
           63     город Боготол         

In [57]:
df_to_send = df.merge(df2, how='inner', left_on=df['municipality'], right_on=df2['caption_full'])
df_to_send

,key_0,municipality,date_event,type_flood,road,f_location,f_road_l,f_road_q,f_water_level,f_closing_date,f_opening_date,oper_mode,f_detour,danger_icon,caption_full,municipality_id,user_id
0,Партизанский район,Партизанский район,28.11.2024,Иное,Партизанское-Мина,64+060,1.34,1,38,26.11.2024,None,Чрезвычайная ситуация,Да,🔴,Партизанский район,47,964635576
1,Партизанский район,Партизанский район,28.11.2024,Иное,Партизанское-Мина,64+060,1.34,1,38,26.11.2024,None,Чрезвычайная ситуация,Да,🔴,Партизанский район,47,5685044175
2,Партизанский район,Партизанский район,26.11.2024,Иное,Партизанское-Мина,64+060,1.34,1,40,26.11.2024,None,Чрезвычайная ситуация,Да,🔴,Партизанский район,47,964635576
3,Партизанский район,Партизанский район,26.11.2024,Иное,Партизанское-Мина,64+060,1.34,1,40,26.11.2024,None,Чрезвычайная ситуация,Да,🔴,Партизанский район,47,5685044175
4,Партизанский район,Партизанский район,27.11.2024,Иное,Партизанское-Мина,64+060,1.34,1,40,26.11.2024,None,Чрезвычайная ситуация,Да,🔴,Партизанский район,47,964635576
5,Партизанский район,Партизанский район,27.11.2024,Иное,Партизанское-Мина,64+060,1.34,1,40,26.11.2024,None,Чрезвычайная ситуация,Да,🔴,Партизанский район,47,5685044175
6,Партизанский район,Партизанский район,29.11.2024,Иное,Партизанское-Мина,64+060,1.34,1,15,26.11.2024,None,Чрезвычайная ситуация,Да,🔴,Партизанский район,47,964635576
7,Партизанский район,Партизанский район,29.11.2024,Иное,Партизанское-Мина,64+060,1.34,1,15,26.11.2024,None,Чрезвычайная ситуация,Да,🔴,Партизанский район,47,5685044175
8,Партизанский район,Партизанский район,02.12.2024,Иное,Партизанское-Мина,64+060,0.30,1,16,26.11.2024,None,Чрезвычайная ситуация,Да,🔴,Партизанский район,47,964635576
9,Партизанский район,Партизанский район,02.12.2024,Иное,Партизанское-Мина,64+060,0.30,1,16,26.11.2024,None,Чрезвычайная ситуация,Да,🔴,Партизанский район,47,5685044175
